In [9]:
# Import required modules
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import chromedriver_autoinstaller
import tldextract
import random

from random import randint
from time import sleep

pd.set_option("display.max_columns", None)
from tqdm import tqdm

tqdm.pandas()

#from skimpy import skim

import pickle

In [10]:
# Selenium headless chrome driver

options = Options()
options.add_argument("--headless=new")
options.add_argument("user-agent=INDIAN_UNIVERSITIES")

# block file downloads
prefs = {"download_restrictions": 3}
options.add_experimental_option("prefs", prefs)

# Initialize the Selenium WebDriver
driver = webdriver.Chrome(options=options)

# driver.set_page_load_timeout(60)  # 60 seconds timeout for page load only if needed

In [11]:
# Extract domain name from URL
def extract_domain(url):
    extracted = tldextract.extract(url)
    # This will combine the domain and the top-level domain (TLD)
    return f"{extracted.domain}.{extracted.suffix}"

In [81]:
with open("homepage_urls_usa.pkl", "rb") as file:
     all_urls = pickle.load(file)[22021:]

In [12]:
with open("homepage_urls_usa2.txt", "rb") as file:
    all_urls = [url.decode('utf-8') if isinstance(url, bytes) else url for url in all_urls]

In [13]:
def extract_page_data(url):
    """Extracts page title and cleaned text from the given URL."""
    try:
        # WebDriver load page
        driver.get(url)

        # Create BeautifulSoup object and specify the parser
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract title of the page
        title = soup.title.string if soup.title else "No title"

        # Extract all text from the page
        raw_text = soup.get_text()
        # Clean the text by replacing multiple white spaces and line breaks with single space
        cleaned_text = " ".join(raw_text.split())

        return title, cleaned_text
    except TimeoutException:
        print(f"Timeout exception for URL: {url}")
        return ("No Data", "No Data")
    except Exception as e:
        print(f"Error for URL {url}: {repr(e)}")
        return ("No Data", "No Data")

# Initialize an empty list to hold the data
data = []

# For each URL in the list of URLs
for url in tqdm(all_urls):
    title, cleaned_text = extract_page_data(url)
    website = extract_domain(url)
    data.append([website, url, title, cleaned_text])

# Also save as a Parquet file as it takes less space on server
# df.to_parquet("scraped_homepage_urls_india.parquet", engine="pyarrow")

100%|█████████████████████████████████████████| 226/226 [07:20<00:00,  1.95s/it]

Error for URL 
: InvalidArgumentException()


In [14]:
# Convert the data into a pandas DataFrame
df = pd.DataFrame(data, columns=["website", "url", "title", "text"])

# Save as CSV file
df.to_csv(
    "usa-bis.csv",
    encoding="utf-8",
    index=False,
)